In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [3]:
sample_set = pd.read_table('../sample_test_set_1/sample_ts1_input.txt', names = ['Test'])

In [4]:
sample_set.info

<bound method DataFrame.info of                                                   Test
0                                                    1
1                                                    0
2    0011101000101010000010000001000101010000000011...
3    0101000010100010000000100100000101000100000111...
4    1101110011111110000111100101111111011100000111...
..                                                 ...
97   1111001010110110000010101100100100100100000101...
98   1111101110110110000110000100000001000100000100...
99   1111001110111110000100001101100101001100000111...
100  1111101111111111110111101101100101111100001111...
101  1111111111111110010100101101100101011110000110...

[102 rows x 1 columns]>

In [5]:
sample_set.head()

,Test
0,1
1,0
2,0011101000101010000010000001000101010000000011...
3,0101000010100010000000100100000101000100000111...
4,1101110011111110000111100101111111011100000111...


In [6]:
# Prvý riadok vstupu udáva počet testovacích prípadov T. 
T = sample_set.Test[0]

In [7]:
# Druhý riadok vstupu udáva percento testovacích prípadov P, na ktoré musíte správne odpovedať, 
# aby sa vaše riešenie považovalo za správne.
P = sample_set.Test[1]

In [8]:
n = 2
test_cases = sample_set.drop(index = sample_set.index[:n])

In [9]:
test_cases.info

<bound method DataFrame.info of                                                   Test
2    0011101000101010000010000001000101010000000011...
3    0101000010100010000000100100000101000100000111...
4    1101110011111110000111100101111111011100000111...
5    1001000001100110000100000001000101000100000101...
6    1111101011111110000111001101110111110101010111...
..                                                 ...
97   1111001010110110000010101100100100100100000101...
98   1111101110110110000110000100000001000100000100...
99   1111001110111110000100001101100101001100000111...
100  1111101111111111110111101101100101111100001111...
101  1111111111111110010100101101100101011110000110...

[100 rows x 1 columns]>

In [10]:
test_cases.head()

,Test
2,0011101000101010000010000001000101010000000011...
3,0101000010100010000000100100000101000100000111...
4,1101110011111110000111100101111111011100000111...
5,1001000001100110000100000001000101000100000101...
6,1111101011111110000111001101110111110101010111...


In [11]:
# Keď sa hráč i pokúsi odpovedať na otázku j, pravdepodobnosť, 
# že na ňu odpovie správne, je f( Si − Qj ), kde f je sigmoidná funkcia:
import numpy as np

def sigmoid(x):
    return 1/(1 + np.exp(-x))

In [12]:
def fair_probability(si, qj):
    return sigmoid(si - qj)

In [13]:
## Hráč i má úroveň zručností Si a otázka j má úroveň obtiažnosti Qj.
# každá obtiažnosť otázky sa vyberá jednotne a náhodne z rozsahu [−3.00, 3.00] 
decision_array = [-3.00, 3.00]

In [14]:
# a nezávisle od všetkých ostatných možností. 
Si = np.random.uniform(decision_array[0], decision_array[1])
Qj = np.random.uniform(decision_array[0], decision_array[1])
Si, Qj

(2.032919345637673, -2.1371936450865032)

In [15]:
fair_probability(Si, Qj)

0.9847845719279239

In [16]:
unfair_probability = 0.5

In [17]:
def split(word):
    return [char for char in word]

In [18]:
test_cases.iloc[0]

Test    0011101000101010000010000001000101010000000011...
Name: 2, dtype: object

In [19]:
%%time

row_lengts = 10000
Qjs = []
for i in range(row_lengts):
    Qjs.append(np.random.uniform(decision_array[0], decision_array[1]))

diff_vals = []
for index, row in test_cases.iterrows():
    np_array = np.array(split(row['Test'])).astype('int32')
    const_of_diff = 0
    Si = np.random.uniform(decision_array[0], decision_array[1])
    
    for i in range(len(np_array)):
        proba = fair_probability(Si, Qjs[i])
        response = 0
        if proba > 0.5:
            response = 1
        
        # print('index,', index, 'Si', Si, 'Qj', Qjs[i], 'probability', proba, 'our_prediction', response, 'real_val', np_array[i])
        if np_array[i] != response:
            const_of_diff = const_of_diff + 1
            
    diff_vals.append(const_of_diff)
    

CPU times: user 3.25 s, sys: 6.47 ms, total: 3.26 s
Wall time: 3.26 s


In [20]:
print(diff_vals)
print(np.max(diff_vals))
print(np.min(diff_vals))

[4218, 7661, 4441, 3069, 4350, 4942, 6404, 4911, 3781, 6552, 6381, 3763, 7358, 7274, 5773, 7718, 6400, 7766, 4585, 5189, 3486, 4911, 5879, 6104, 4704, 5129, 5352, 4420, 2380, 6280, 4925, 6117, 5202, 6927, 5732, 3718, 6335, 5295, 4341, 4639, 2253, 6242, 3867, 5094, 5943, 3170, 4823, 2131, 5211, 7309, 5083, 4647, 5094, 4722, 4964, 6628, 4660, 5530, 1609, 6321, 4910, 4746, 3325, 6577, 1816, 3949, 6631, 5780, 5482, 2153, 6350, 5060, 4303, 5358, 4290, 7789, 5671, 4058, 6685, 3584, 3795, 5885, 8058, 4471, 4145, 5231, 5049, 4725, 3160, 6143, 5204, 3757, 5181, 8133, 5412, 6559, 2900, 4161, 6620, 4791]
8133
1609


In [21]:
%%time
for i in range(len(diff_vals)):
    if diff_vals[i] > 7500:
        print(i)

1
15
17
75
82
93
CPU times: user 144 µs, sys: 54 µs, total: 198 µs
Wall time: 175 µs


In [72]:
# %%time
# ## Bootstrapping
# bootstrapped_attempts = []
# for j in range(300):
#     row_lengths = 10000
#     Qjs = []
#     for i in range(row_lengts):
#         Qjs.append(np.random.uniform(decision_array[0], decision_array[1]))
    
#     diff_vals = []
#     for index, row in test_cases.iterrows():
#         np_array = np.array(split(row['Test'])).astype('int32')
#         const_of_diff = 0
#         Si = np.random.uniform(decision_array[0], decision_array[1])

#         for x in range(row_lengths):
#             proba = fair_probability(Si, Qjs[x])
#             response = 0
#             if proba > 0.5:
#                 response = 1
#             if np_array[x] != response:
#                 const_of_diff = const_of_diff + 1

#         diff_vals.append(const_of_diff)
        
#     max_value = max(diff_vals)
#     max_index = diff_vals.index(max_value)
#     bootstrapped_attempts.append(max_index)

CPU times: user 17min 43s, sys: 885 ms, total: 17min 44s
Wall time: 17min 44s


In [22]:
def calculate_differences(data_frame):
    row_lengths = 10000
    Qjs = [np.random.uniform(decision_array[0], decision_array[1]) for i in range(row_lengths)]
    diff_vals = []
    for index, row in data_frame.iterrows():
        const_of_diff = 0
        Si = np.random.uniform(decision_array[0], decision_array[1])

        responses = np.array(split(row['Test'])).astype('int32')
        sim_responses = np.array([fair_probability(Si, Qjs[i]) > 0.5 for i in range(row_lengths)])

        for (sim_res, response) in zip(sim_responses, responses):
            if sim_res != response:
                const_of_diff += 1

        diff_vals.append(const_of_diff)

    max_value = max(diff_vals)
    return diff_vals.index(max_value)


In [ ]:
%%time
## Bootstrapping
bootstrapped_attempts = []
for j in range(500):
    val = calculate_differences(test_cases)
    bootstrapped_attempts.append(val)

In [ ]:
bootstrapped_attempts

In [ ]:
plt.hist(bootstrapped_attempts);

In [ ]:
counters = {}
for i in range(len(bootstrapped_attempts)):
    if bootstrapped_attempts[i] in counters:
        counters[bootstrapped_attempts[i]] = counters[bootstrapped_attempts[i]] + 1
    else:
        counters[bootstrapped_attempts[i]] = 1
        
counters

In [ ]:
max_value = max(counters, key=counters.get)
print(max_value)

In [25]:
# asi riesenie 58 + 1: 42 -> pri 200 pokusoch najvaic krat
with open('output_1.txt', 'w') as f:
    f.write('Case #1: 59')